<a href="https://colab.research.google.com/github/soyoonjeong/Running_Machine/blob/main/%EC%84%B1%EB%8F%99%EA%B5%AC_%ED%97%AC%EC%8A%A4%EC%9E%A5_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import requests
url = "http://openapi.seoul.go.kr:8088/46655979586a656e313232736d506b79/xml/LOCALDATA_104201_SD/1/200/"

response = requests.get(url)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\n<LOCALDATA_104201_SD>\n<list_total_count>146</list_total_count>\n<RESULT>\n<CODE>INFO-000</CODE>\n<MESSAGE>\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4</MESSAGE>\n</RESULT>\n<row>\n<OPNSFTEAMCODE>3030000</OPNSFTEAMCODE>\n<MGTNO>CDFH3301061989000001</MGTNO>\n<APVPERMYMD>19891024</APVPERMYMD>\n<APVCANCELYMD/>\n<TRDSTATEGBN>03</TRDSTATEGBN>\n<TRDSTATENM>\xed\x8f\x90\xec\x97\x85</TRDSTATENM>\n<DTLSTATEGBN>03</DTLSTATEGBN>\n<DTLSTATENM>\xed\x8f\x90\xec\x97\x85</DTLSTATENM>\n<DCBYMD>20060116</DCBYMD>\n<CLGSTDT/>\n<CLGENDDT/>\n<ROPNYMD/>\n<SITETEL>466-9845</SITETEL>\n<SITEAREA/>\n<SITEPOSTNO>133837</SITEPOSTNO>\n<SITEWHLADDR>\xec\x84\x9c\xec\x9a\xb8\xed\x8a\xb9\xeb\xb3\x84\xec\x8b\x9c \xec\x84\xb1\xeb\x8f\x99\xea\xb5\xac \xec\x86\xa1\xec\xa0\x95\xeb\x8f\x99 87-17\xeb\xb2\x88\xec\xa7\x80 </SITEWHLADDR>\n<RDNWHLADDR>\xec\x84\x9c\xec\x9a\xb8\xed\x8a\xb9\xeb\xb3\x84\xec\x8b\x9c \xec\x84\xb1\

In [ ]:
pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def make_dataframe(url):
  response = requests.get(url)
  df = pd.read_xml(response.content)
  df.drop(df.index[[0,1]], inplace = True) # 0부터 두 행 없애기
  df = df[df['TRDSTATEGBN']==1] # 정상영업만
  df = df.reset_index() # index reset
  df = df.drop(['index'], axis =1)
  df = df[['DTLSTATENM','SITETEL','RDNWHLADDR','BPLCNM','X','Y']] # column 정리
  df.columns = ['open','loc_number','loc_address','loc_name','X','Y'] # column 이름 바꾸기
  # 위도 경도로 바꿔주기
  from pyproj import Proj, transform
  proj_1 = Proj(init='epsg:2097' )# 중부원점(Bessel): 서울 등 중부지역 EPSG:2097
  proj_2 = Proj(init='epsg:4326') # WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326
  converted = transform(proj_1, proj_2, df['X'].values, df['Y'].values)
  df['loc_lng'] = converted[0]
  df['loc_lat'] = converted[1]
  df.drop(['X','Y'], axis = 1, inplace=True)
  return df

In [ ]:
df = make_dataframe(url)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


In [ ]:
df.head()

,open,loc_number,loc_address,loc_name,loc_lng,loc_lat
0,영업중,2298-7071,"서울특별시 성동구 독서당로 272, 상가동 201호 (금호동4가, 금호동 대우아파트)",AII For U GYM,127.016793,37.545973
1,영업중,02-2299-0044,"서울특별시 성동구 독서당로 166, B201호 (옥수동, 성원상떼뷰)",(주)옥수 투엑스 휘트니스,127.008734,37.540707
2,영업중,469-0067,"서울특별시 성동구 뚝섬로 400, 뚝섬쇼핑센타 3층 (성수동2가)",올림픽휘트니스,127.052078,37.538473
3,영업중,0507-1349-5118,"서울특별시 성동구 성덕정길 106, 승재빌딩 4층 (성수동2가)",스타트 짐,127.052205,37.537100
4,영업중,461-9901,"서울특별시 성동구 성수일로 39, 경동빌딩 지층 (성수동1가)",짐아서 휘트니스,127.046208,37.544323


In [ ]:
df.isnull().sum()

open            0
loc_number     29
loc_address     0
loc_name        0
loc_lng         0
loc_lat         0
dtype: int64

## 함수 내용

In [ ]:
df = pd.read_xml(response.content)
df

,list_total_count,CODE,MESSAGE,OPNSFTEAMCODE,MGTNO,APVPERMYMD,APVCANCELYMD,TRDSTATEGBN,TRDSTATENM,DTLSTATEGBN,...,Y,CULPHYEDCOBNM,PUPRSENM,INSURJNYNCODE,LDERCNT,BDNGDNGNUM,BDNGYAREA,MEMCOLLTOTSTFNUM,DRMKCOBNM,BUPNM
0,146.0,None,None,NaN,None,NaN,NaN,NaN,None,NaN,...,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,INFO-000,정상 처리되었습니다,NaN,None,NaN,NaN,NaN,None,NaN,...,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,None,None,3030000.0,CDFH3301061989000001,19891024.0,NaN,3.0,폐업,3.0,...,450279.646798,체력단련장업,사립,0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,None,None,3030000.0,CDFH3301061989000002,19891222.0,NaN,3.0,폐업,3.0,...,451267.516899,체력단련장업,사립,0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,None,None,3030000.0,CDFH3301061989000003,19891229.0,NaN,4.0,취소/말소/만료/정지/중지,35.0,...,451570.344199,체력단련장업,사립,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,NaN,None,None,3030000.0,CDFH3301062022000007,20220527.0,NaN,1.0,영업/정상,13.0,...,450154.454197,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
144,NaN,None,None,3030000.0,CDFH3301062022000008,20220704.0,NaN,1.0,영업/정상,13.0,...,451228.899925,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
145,NaN,None,None,3030000.0,CDFH3301062022000009,20220705.0,NaN,1.0,영업/정상,13.0,...,448366.505563,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
146,NaN,None,None,3030000.0,CDFH3301062022000010,20220708.0,NaN,1.0,영업/정상,13.0,...,449179.325706,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[['X','Y']]

,X,Y
0,NaN,NaN
1,NaN,NaN
2,206188.003773,450279.646798
3,202727.017019,451267.516899
4,204590.476506,451570.344199
...,...,...
143,201632.950718,450154.454197
144,202797.681704,451228.899925
145,204667.627242,448366.505563
146,204727.717641,449179.325706


In [ ]:
list_total_count = df.iloc[0,0] # 해당 구 안에 헬스장 개수
df.drop(df.index[[0,1]], inplace = True) # 0부터 두 행 없애기

In [ ]:
df = df[df['TRDSTATEGBN']==1] # 정상영업만

In [ ]:
df = df.reset_index() # index reset
df.head()

,index,list_total_count,CODE,MESSAGE,OPNSFTEAMCODE,MGTNO,APVPERMYMD,APVCANCELYMD,TRDSTATEGBN,TRDSTATENM,...,Y,CULPHYEDCOBNM,PUPRSENM,INSURJNYNCODE,LDERCNT,BDNGDNGNUM,BDNGYAREA,MEMCOLLTOTSTFNUM,DRMKCOBNM,BUPNM
0,32,NaN,None,None,3030000.0,CDFH3301062000000005,20001227.0,NaN,1.0,영업/정상,...,449301.451523,체력단련장업,사립,None,NaN,NaN,NaN,NaN,NaN,NaN
1,41,NaN,None,None,3030000.0,CDFH3301062004000001,20040524.0,NaN,1.0,영업/정상,...,448716.877164,체력단련장업,사립,None,NaN,1.0,NaN,NaN,NaN,NaN
2,43,NaN,None,None,3030000.0,CDFH3301062004000004,20040605.0,NaN,1.0,영업/정상,...,448470.237859,체력단련장업,사립,None,2.0,1.0,NaN,NaN,NaN,NaN
3,46,NaN,None,None,3030000.0,CDFH3301062004000007,20041004.0,NaN,1.0,영업/정상,...,448317.872400,체력단련장업,사립,None,NaN,1.0,NaN,NaN,NaN,NaN
4,49,NaN,None,None,3030000.0,CDFH3301062005000001,20050224.0,NaN,1.0,영업/정상,...,449119.290164,체력단련장업,사립,None,1.0,NaN,3002.07,NaN,NaN,NaN


In [ ]:
df = df.drop(['index'], axis =1)

In [ ]:
df = df[['DTLSTATENM','SITETEL','SITEWHLADDR','RDNWHLADDR','BPLCNM','X','Y']] # column 정리

In [ ]:
df.columns = ['open','loc_number','loc_address1','loc_address2','loc_name','X','Y'] # column 이름 바꾸기

In [ ]:
df.head()

,open,loc_number,loc_address1,loc_address2,loc_name,X,Y
0,영업중,2298-7071,서울특별시 성동구 금호동4가 800 대우(아)상가 201호,"서울특별시 성동구 독서당로 272, 상가동 201호 (금호동4가, 금호동 대우아파트)",AII For U GYM,201669.803557,449301.451523
1,영업중,02-2299-0044,서울특별시 성동구 옥수동 474,"서울특별시 성동구 독서당로 166, B201호 (옥수동, 성원상떼뷰)",(주)옥수 투엑스 휘트니스,200957.650467,448716.877164
2,영업중,469-0067,서울특별시 성동구 성수동2가 339-5 뚝섬쇼핑센타,"서울특별시 성동구 뚝섬로 400, 뚝섬쇼핑센타 3층 (성수동2가)",올림픽휘트니스,204788.453049,448470.237859
3,영업중,0507-1349-5118,서울특별시 성동구 성수동2가 561-1 국민은행,"서울특별시 성동구 성덕정길 106, 승재빌딩 4층 (성수동2가)",스타트 짐,204799.750079,448317.872400
4,영업중,461-9901,서울특별시 성동구 성수동1가 22-5 경동빌딩,"서울특별시 성동구 성수일로 39, 경동빌딩 지층 (성수동1가)",짐아서 휘트니스,204269.337017,449119.290164


In [ ]:
pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 5.2 MB/s 


In [ ]:
from pyproj import Proj, transform
# 위도 경도로 바꿔주기
# Projection 정의
# 중부원점(Bessel): 서울 등 중부지역 EPSG:2097
proj_1 = Proj(init='epsg:2097')
# WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326
proj_2 = Proj(init='epsg:4326')
converted = transform(proj_1, proj_2, df['X'].values, df['Y'].values)
df['loc_lng'] = converted[0]
df['loc_lat'] = converted[1]
df.drop(['X','Y'], axis = 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


NameError: ignored

## 주소 -> 위도 경도
https://parkgihyeon.github.io/project/geocoding-api/

In [ ]:
import requests, json

def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
  # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
  headers = {"Authorization": "KakaoAK 07d87cd8a22ca8cdf61219221e33eb78"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"lat": str(address['y']), "lng": str(address['x'])}

  return crd

crd = get_location("서울특별시 동작구 동작대로25길 16 CS빌딩 6.7층")
print(crd)
# for i in range(len(df)):
#   try:
#     crd = get_location(df['address'][i])
#   except:
#     crd = get_location(df['address2'][i])
#   df['lat'][i], df['lng'][i] = crd['lat'], crd['lng']
#   print(df['name'][i], crd)

{'lat': '37.4863323127883', 'lng': '126.980927405478'}


In [ ]:
import requests, json, pprint

def get_address(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+lng+"&y="+lat
    # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
    # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
    headers = {"Authorization": "KakaoAK 07d87cd8a22ca8cdf61219221e33eb78"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)

    return full_address

full_address = get_address('37.5453300340479', '127.017925368592')
pprint.pprint(full_address)

{'documents': [{'address_name': '서울특별시 성동구 금호동4가',
                'code': '1120011200',
                'region_1depth_name': '서울특별시',
                'region_2depth_name': '성동구',
                'region_3depth_name': '금호동4가',
                'region_4depth_name': '',
                'region_type': 'B',
                'x': 127.02616193500656,
                'y': 37.54473257558382},
               {'address_name': '서울특별시 성동구 금호4가동',
                'code': '1120062000',
                'region_1depth_name': '서울특별시',
                'region_2depth_name': '성동구',
                'region_3depth_name': '금호4가동',
                'region_4depth_name': '',
                'region_type': 'H',
                'x': 127.02240132684157,
                'y': 37.54719127436341}],
 'meta': {'total_count': 2}}


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         83 non-null     object 
 1   loc_number   55 non-null     object 
 2   loc_address  83 non-null     object 
 3   loc_name     83 non-null     object 
 4   loc_lng      83 non-null     float64
 5   loc_lat      83 non-null     float64
dtypes: float64(2), object(4)
memory usage: 4.0+ KB


전화번호 앞에 '02-' 붙일까 말까

# 데이터 크롤링

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 30.9 MB/s 
     |████████████████████████████████| 358 kB 52.3 MB/s 
     |████████████████████████████████| 139 kB 54.1 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 4.1 MB 49.6 MB/s 
     |████████████████████████████████| 58 kB 4.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [87.8 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,905 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
# 웹 드라이버 로딩
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome("chromedriver", chrome_options=chrome_options)
driver.implicitly_wait(3) # driver가 로딩될 때까지 기다리기 위해 implicitly_wait(3) 줌

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
df['naver_map_url'] = 'none'

## 헬스장 url 추출

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/seongdonggu_gym_3.csv")
df = df[['id','name','img','grade','lat','lng','phoneNumber','address']]
df.head()

,id,name,img,grade,lat,lng,phoneNumber,address
0,1150285601,올포유짐,https://search.pstatic.net/common/?autoRotate=...,5.00,37.545973,127.016793,2298-7071,"서울특별시 성동구 독서당로 272, 상가동 201호 (금호동4가, 금호동 대우아파트)"
1,272888535,투엑스휘트니스 옥수점,https://search.pstatic.net/common/?autoRotate=...,4.92,37.540707,127.008734,02-2299-0044,"서울특별시 성동구 독서당로 166, B201호 (옥수동, 성원상떼뷰)"
2,18788499,올림픽휘트니스,https://search.pstatic.net/common/?autoRotate=...,100.00,37.538473,127.052078,469-0067,"서울특별시 성동구 뚝섬로 400, 뚝섬쇼핑센타 3층 (성수동2가)"
3,32085717,스타트짐헬스,https://search.pstatic.net/common/?autoRotate=...,100.00,37.537100,127.052205,0507-1349-5118,"서울특별시 성동구 성덕정길 106, 승재빌딩 4층 (성수동2가)"
4,1530150594,짐아서,https://search.pstatic.net/common/?autoRotate=...,100.00,37.544323,127.046208,461-9901,"서울특별시 성동구 성수일로 39, 경동빌딩 지층 (성수동1가)"


In [ ]:
df[df['name']=='바른핏 PT&헬스 옥수점']

,id,name,img,grade,lat,lng,phoneNumber,address
69,1646071896,바른핏 PT&헬스 옥수점,https://search.pstatic.net/common/?autoRotate=...,100.0,37.544034,127.012912,0222929722,"서울특별시 성동구 독서당로 226, 조을빌딩 2층 (옥수동)"


In [ ]:
!pip install geopy
from geopy.geocoders import Nominatim

#데이터의 좌표주소 열을 이용해 지오코딩
for i, loc in enumerate(df['loc_address1']):
    print(loc)
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(loc)
    print(geo)
    df.loc[i, 'X'] = str(geo.latitude)
    df.loc[i, 'Y'] = str(geo.longitude)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
서울특별시 성동구 송정동 87-17번지
None


AttributeError: ignored

In [ ]:
df = df[['id','name','img','grade','lat','lng','phoneNumber','address']]

In [ ]:
df.head()

,id,name,img,grade,lat,lng,phoneNumber,address,address2
0,1150285601,올포유짐,https://search.pstatic.net/common/?autoRotate=...,5.00,37.545330,127.017925,2298-7071,"서울특별시 성동구 독서당로 272, 상가동 201호 (금호동4가, 금호동 대우아파트)",서울 성동구 독서당로 270 대우아파트상가 201호
1,272888535,투엑스휘트니스 옥수점,https://search.pstatic.net/common/?autoRotate=...,4.92,37.540606,127.011698,02-2299-0044,"서울특별시 성동구 독서당로 166, B201호 (옥수동, 성원상떼뷰)",서울 성동구 독서당로 166 성원상떼뷰 1F/B1/B2
2,18788499,올림픽휘트니스,https://search.pstatic.net/common/?autoRotate=...,100.00,37.538509,127.054999,469-0067,"서울특별시 성동구 뚝섬로 400, 뚝섬쇼핑센타 3층 (성수동2가)",서울 성동구 뚝섬로 400 뚝섬쇼핑센타
3,32085717,스타트짐헬스,https://search.pstatic.net/common/?autoRotate=...,100.00,37.537159,127.055105,0507-1349-5118,"서울특별시 성동구 성덕정길 106, 승재빌딩 4층 (성수동2가)",서울 성동구 성덕정길 106
4,1530150594,짐아서,https://search.pstatic.net/common/?autoRotate=...,100.00,37.544237,127.049256,461-9901,"서울특별시 성동구 성수일로 39, 경동빌딩 지층 (성수동1가)",서울 성동구 성수일로 39 경동빌딩 B1


In [ ]:
df.rename(columns={'naver_map_url':'id'},inplace=True)
df.rename(columns={'loc_name':'name'},inplace=True)
df.rename(columns={'loc_lat':'lat'},inplace=True)
df.rename(columns={'loc_lng':'lng'},inplace=True)
df.rename(columns={'loc_number':'phoneNumber'},inplace=True)
df.rename(columns={'loc_address':'address'},inplace=True)

In [ ]:
df.rename(columns={'company_id':'id'},inplace=True)

In [ ]:
for i, keyword in enumerate(df['loc_name'].tolist()):
  print("이번에 찾을 키워드 : ", keyword)
  try:
    if df['naver_map_url'][i]!='none': # 이미 url 있는 거 다 패스
      continue
    address_idx_start = df['loc_address'][i].index("(")+1
    address_idx_end =  len(df['loc_address'][i])-df['loc_address'][i][::-1].index("동") # 동 이름 추출
    naver_map_search_url = "https://map.naver.com/v5/search/"+"성동구"+df['loc_address'][i][address_idx_start:address_idx_end]+keyword+'/place' # 구 + 동 + 헬스장 이름
    print(naver_map_search_url, end=' ')

    driver.get(naver_map_search_url)
    time.sleep(12)
    cu = driver.current_url
    res_code = re.findall(r"place/(\d+)", cu)
    final_url = 'https://pcmap.place.naver.com/place/'+res_code[0]+'/review/visitor#'
    print(keyword, final_url)
    df['naver_map_url'][i] = final_url
  except IndexError:
    df['naver_map_url'][i] = 'none'
    print('url을 찾지 못했습니당')

이번에 찾을 키워드 :  올포유짐
이번에 찾을 키워드 :  투엑스휘트니스 옥수점
이번에 찾을 키워드 :  올림픽휘트니스
이번에 찾을 키워드 :  스타트짐헬스
이번에 찾을 키워드 :  짐아서
이번에 찾을 키워드 :  라온휘트니스&필라테스 마장역점
이번에 찾을 키워드 :  웰트짐 성수점
이번에 찾을 키워드 :  동우휘트니스
이번에 찾을 키워드 :  시티짐 답십리
이번에 찾을 키워드 :  포시즌휘트니스 왕십리점
이번에 찾을 키워드 :  팀키스짐
이번에 찾을 키워드 :  노블휘트니스
이번에 찾을 키워드 :  고릴라멀티짐 행당점
이번에 찾을 키워드 :  스포애니 왕십리역점
이번에 찾을 키워드 :  센트리얼휘트니스 옥수점
이번에 찾을 키워드 :  파크짐 행당점
이번에 찾을 키워드 :  스타트짐
이번에 찾을 키워드 :  파크짐 행당점
이번에 찾을 키워드 :  파크짐 행당점
이번에 찾을 키워드 :  MSC 운동센터
이번에 찾을 키워드 :  에코휘트니스
이번에 찾을 키워드 :  건강과땀 한양대점
이번에 찾을 키워드 :  인피니티퍼스널트레이닝
이번에 찾을 키워드 :  피네테스 옥수점
이번에 찾을 키워드 :  허니바디짐
이번에 찾을 키워드 :  BAN(반)
이번에 찾을 키워드 :  레이 휘트니스
이번에 찾을 키워드 :  바른몸 PT STUDIO
이번에 찾을 키워드 :  피트니스 에이치(Fitness. H)
이번에 찾을 키워드 :  센트라스 투엑스 주식회사
이번에 찾을 키워드 :  커브스 금호클럽
이번에 찾을 키워드 :  테라피티
이번에 찾을 키워드 :  위드짐 금호점(WITHGYM)
이번에 찾을 키워드 :  메디앤핏
이번에 찾을 키워드 :  (주)넉아웃
이번에 찾을 키워드 :  탑핏(TOPFIT)
이번에 찾을 키워드 :  퀄핏 건강운동센터
이번에 찾을 키워드 :  크로스핏 핏불
이번에 찾을 키워드 :  무브먼트 프로젝트
이번에 찾을 키워드 :  비 스튜디오(Be studio)
이번에 찾을 키워드 :  나의 근육 사용설명서
이번에 찾을 키워드 :  파크짐 한양대점
이번에 찾을 키워

In [ ]:
df[df['naver_map_url']=='none'].count()

Unnamed: 0       0
open             0
loc_number       0
loc_address      0
loc_name         0
loc_lng          0
loc_lat          0
naver_map_url    0
dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/seongdonggu_company_new.csv")

In [ ]:
df[df['naver_map_url']=='none']

,open,loc_number,loc_address,loc_name,loc_lng,loc_lat,naver_map_url
27,영업중,02.2297-6865,"서울특별시 성동구 금호로 100, 상가동 210~216호 (금호동1가, 벽산아파트)",BAN(반),127.024226,37.551857,none
31,영업중,02-499-9686,"서울특별시 성동구 동일로 273, 경한빌딩 2,3층 (송정동)",피트니스 에이치(Fitness. H),127.068562,37.555908,none
36,영업중,0222969690,"서울특별시 성동구 독서당로 344, 힐스테이트 금호 상가동 B1층 20-8호, 20...",위드짐 금호점(WITHGYM),127.022660,37.545619,none
38,영업중,None,"서울특별시 성동구 서울숲2길 18-11, 2층 (성수동1가)",(주)넉아웃,127.037734,37.546822,none
49,영업중,None,"서울특별시 성동구 왕십리로 115, 9층 905호 (성수동1가)",(주)프롬더바디,127.041295,37.547828,none
66,영업중,None,"서울특별시 성동구 고산자로 255, 케이 타워 7층 (도선동)",PT STUDIO RE ' ON,127.032837,37.562619,none
82,영업중,None,"서울특별시 성동구 성덕정길 92, 3층 (성수동2가)",스위치 P.T,127.050710,37.537539,none


In [ ]:
df[df['naver_map_url']=='none']['loc_name']

Series([], Name: loc_name, dtype: object)

In [ ]:
none = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/none.CSV", encoding = 'cp949')
none.head()

,loc_name,url_num
0,AII For U GYM,1150285601
1,(주)옥수 투엑스 휘트니스,272888535
2,짐아서 휘트니스,1530150594
3,시티짐 답십리,34346765
4,풍림휘트니스,0


In [ ]:
for idx, row in none.iterrows():
  print(idx, row.loc_name, row.url_num)
  df.loc[df['loc_name']==row.loc_name,'naver_map_url'] = 'https://pcmap.place.naver.com/place/'+str(row.url_num)+'/review/visitor#'

0 AII For U GYM 1150285601
1 (주)옥수 투엑스 휘트니스 272888535
2 짐아서 휘트니스 1530150594
3 시티짐 답십리 34346765
4 풍림휘트니스 0
5 팀키스 1061867273
6 왕십리역점 스포애니 주)케이디스포츠 1006755563
7 파크짐 36112469
8 파크짐 34287354
9 퀸스테이지 0
10  파크짐 35044371
11 이루어짐 헬스 0
12 센트라스 투엑스 주식회사 73553317
13 D_GYM 0
14 메디앤핏 1025976659
15 탑핏(TOPFIT) 1346017056
16 얼라이브짐 0
17 비 스튜디오(Be studio) 1697698006
18 (주)스포티즌 엑시온점 0
19 (주)운동의 정석 1452177609
20 PND 1146239356
21 퀸 스페셜 PT 0
22 바른피티 1877534839
23 와츠핏(What's fit) 1316396572


In [ ]:
df = df[df['naver_map_url']!='https://pcmap.place.naver.com/place/0/review/visitor#']

In [ ]:
df = df.reset_index() # index reset
df = df.drop(['index'], axis =1)

In [ ]:
df.head(11)

,Unnamed: 0,open,loc_number,loc_address,loc_name,loc_lng,loc_lat,naver_map_url,rating
0,0,영업중,2298-7071,"서울특별시 성동구 독서당로 272, 상가동 201호 (금호동4가, 금호동 대우아파트)",올포유짐,127.016793,37.545973,https://pcmap.place.naver.com/place/1150285601...,5.00
1,1,영업중,02-2299-0044,"서울특별시 성동구 독서당로 166, B201호 (옥수동, 성원상떼뷰)",투엑스휘트니스 옥수점,127.008734,37.540707,https://pcmap.place.naver.com/place/272888535/...,4.92
2,2,영업중,469-0067,"서울특별시 성동구 뚝섬로 400, 뚝섬쇼핑센타 3층 (성수동2가)",올림픽휘트니스,127.052078,37.538473,https://pcmap.place.naver.com/place/18788499/r...,100.00
3,3,영업중,0507-1349-5118,"서울특별시 성동구 성덕정길 106, 승재빌딩 4층 (성수동2가)",스타트짐헬스,127.052205,37.537100,https://pcmap.place.naver.com/place/32085717/r...,100.00
4,4,영업중,461-9901,"서울특별시 성동구 성수일로 39, 경동빌딩 지층 (성수동1가)",짐아서,127.046208,37.544323,https://pcmap.place.naver.com/place/1530150594...,100.00
5,5,영업중,02-2282-6027,"서울특별시 성동구 마장로 302, 지하1,2층 (마장동)",라온휘트니스&필라테스 마장역점,127.040343,37.565924,https://pcmap.place.naver.com/place/21111585/r...,100.00
6,6,영업중,461-0828,"서울특별시 성동구 연무장5가길 21, 홍일빌딩 지하2층 (성수동2가)",웰트짐 성수점,127.052193,37.543500,https://pcmap.place.naver.com/place/11691712/r...,4.45
7,7,영업중,NaN,"서울특별시 성동구 천호대로 296, 5층 (용답동)",동우휘트니스,127.050258,37.565980,https://pcmap.place.naver.com/place/37309187/r...,4.05
8,8,영업중,02-2245-0252,"서울특별시 성동구 천호대로 308, 인암빌딩 지하2층 (용답동)",시티짐 답십리점,127.051346,37.565229,https://pcmap.place.naver.com/place/34346765/r...,100.00
9,9,영업중,02-2292-6886,서울특별시 성동구 왕십리광장로 17 (행당동),포시즌휘트니스 왕십리점,127.035160,37.563684,https://pcmap.place.naver.com/place/37638105/r...,4.50


In [ ]:
df = df[['open','loc_number','loc_address','loc_name','loc_lng','loc_lat','naver_map_url','grade']]

## 헬스장 방문자 리뷰 크롤링

In [ ]:
review_json = {}

In [ ]:
import json

with open('/content/drive/MyDrive/Colab Notebooks/seongdonggu_review.json', 'r',encoding='UTF8') as f:
  review_json =  json.load(f)
print(json.dumps(review_json,indent='\t',ensure_ascii=False))

{
	"올포유짐": [
		"주변에 추천하고 싶은 헬스장입니다. 친절하게 맞아주시고 상담도 자세히 해주셔서 첫인상부터 좋았습니다. 시설 둘러보는데 오픈한지 얼마 안돼서 인테리어, 기구, 샤워실 전부 깨끗하고, 가격도 너무 좋아서 고민 없이 등록했어요. 운동 기깔나게 할 수 있을 것 같아요!",
		"재등록합니다. 운동하기 너무 좋아요~ 강력 추천합니다👍👍",
		"디스크로 고생해서 병원 다니다가 여기서 pt 받아봤는데 전보다 훨씬 좋아졌어요. 몸 상태랑 통증 원인, 근육별 기능, 적합한 기구 등에 대해서도 자세히 설명해주셔서 운동 과정을 이해하면서 할 수 있고 재밌었습니다. ㅎㅎ",
		"PT 선생님들 정말 전문지식 가득하시고 꼼꼼하고 체계적으로 자세 봐주시고 알려주세요! 시설도 늘 깔끔하게 유지되고 너무 만족스럽습니다🤎",
		"집 가까운곳에 이렇게 좋은 헬스장이 있어서 좋네요 운동기구도 엄청다양하고 선생님들이 친절하셔서 상담도 오래잘받고 궁금증도 다 풀렸네요ㅎ 이제 운동열심히해서 이쁜몸 만들일만 남았어요~ 깨끗하고 좋아용",
		"자세 잘봐주시고 시설도 너무 좋고 운동할 맛이 납니다\n앞으로 뼈를 묻겠습니다!!!",
		"선생님들도 친절하시고, 기구도 거의 새것이고, 분위기도 너무 좋습니다!! 운동이 하고 싶어지는 곳입니다!",
		"오픈한지 얼마 되지 않아 깔끔하고 쾌적하네요^^ 친절하게 상담 해주셔서 고민없이 바로 가입했습니다. \n앞으로 여기서 운동할 생각하니까 기대됩니다!",
		"3개월 열심히 다니고 또 등록했습니다! 선생님들 친절하시며 매우 깨끗합니다!! 좋아요!!!",
		"혼자 운동 하다 좋아서 와이프도 추천하여 PT\n운동 시작 하는대 합리적이고 좋아요  "
	],
	"투엑스휘트니스 옥수점": [
		"집 근처에 있어서 이용하기 편하네여!!",
		"굳",
		"굿",
		"굿",
		"굿",
		"좋아요",
		"항상 재등록하는 곳이에요 시설도 크고 관리 잘되는 편이구요 요즘 코로나때매 신경 많이 쓰이는데 이 근처에서 유일한 지상시설이여서

In [ ]:
df.shape

(77, 7)

In [ ]:
review_df = pd.DataFrame()
review_df['company_id'] = df['id']
review_df['review0'] = " "
review_df['review1'] = " "
review_df['review2'] = " "
review_df['review3'] = " "
review_df['review4'] = " "
review_df['review5'] = " "
review_df['review6'] = " "
review_df['review7'] = " "
review_df['review8'] = " "
review_df['review9'] = " "

In [ ]:
for i in range(len(df)):
  print(i,"번째 헬스장")
  thisurl = url = 'https://pcmap.place.naver.com/place/'+str(df['id'][i])+'/review/visitor#'
  print(thisurl)
  driver.get(thisurl)
  time.sleep(5)

  html = driver.page_source
  soup = BeautifulSoup(html, 'lxml')
  time.sleep(1)

  try: # 헬스장 이름 수정
    gym_name_naver = soup.find_all('span',attrs = {'class':'_3XamX'})[0]
    print(gym_name_naver.getText())
    df['name'][i] = gym_name_naver.getText()
  except NoSuchElementException:
    print("헬스장 이름 찾을 수 없습니다.")

  gym_name = df['name'][i]
  print("헬스장 이름 :",gym_name)

  print("리뷰 확인 open!")
  try:
    reviews = soup.select("#app-root > div > div > div > div:nth-of-type(7) > div:nth-of-type(2) > div.place_section._3fSeV > div > ul > li")
    print("찾아진 리뷰 개수 :",len(reviews))
    review_df['company_id'][i] = df['id'][i]
    if len(reviews)==0:
      reviews = soup.select("#app-root > div > div > div > div:nth-of-type(7) > div > div.place_section._3fSeV > div > ul > li") # 이런 점이 좋아요 없을 경우
      if len(reviews)==0 :
        print("리뷰가 없습니다.")
        reviews = []
        continue
  except NoSuchElementException:
    print("리뷰가 없습니다")

  columns = review_df.columns[1:]
  c = 0
  print("리뷰 내용 open!")
  try:
    for r in range(len(reviews)):
      review_content = reviews[r].select("div.faZHB > a > span")
      if len(review_content)>0: # 글 리뷰가 없는 경우 방지
        review_df[columns[c]][i] = review_content[0].getText()
        c+=1
        print("리뷰 내용 :", review_content[0].getText())
        print("-----------------------------------------------------")
    for r in range(10-len(reviews)):
      review_df[columns[c]][i] = " "
      c+=1
  except NoSuchElementException:
    print("리뷰 안찾아짐")


0 번째 헬스장
https://pcmap.place.naver.com/place/1150285601/review/visitor#
올포유짐
헬스장 이름 : 올포유짐
리뷰 확인 open!
찾아진 리뷰 개수 : 10


NameError: ignored

In [ ]:
for i in range(len(df)):
  print(i,"번째 헬스장")
  thisurl = 'https://pcmap.place.naver.com/place/'+str(df['id'][i])+'/review/visitor#'
  print(thisurl)
  driver.get(thisurl)
  time.sleep(5)

  while True:
    try: # 더보기 클릭
      time.sleep(1)
      driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
      time.sleep(3)
      element = driver.find_element(By.CSS_SELECTOR,'div.place_section > div._2kAri > a')
      driver.execute_script("arguments[0].click();", element)
      # driver.find_element(By.CSS_SELECTOR,'div.place_section > div._2kAri > a').send_keys(Keys.ENTER) # 더보기
      time.sleep(3)
      driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
      time.sleep(1)
    except NoSuchElementException:
      print('-더보기 버튼 모두 클릭 완료-')
      break


  html = driver.page_source
  soup = BeautifulSoup(html, 'lxml')
  time.sleep(1)

  gym_name = df['name'][i]
  print("헬스장 이름 :",df['name'][i])
  review_json[df['name'][i]] = []

  print("리뷰 개수 open!")
  try: # 리뷰 개수
    review_num = soup.find_all('span', attrs = {'class':'place_section_count'})[0].getText() # 리뷰 개수
    print("리뷰 개수 :", review_num)
  except IndexError:
    print("리뷰 개수 찾을 수 없습니다.")
    continue

  print("리뷰 확인 open!")
  try:
    reviews = soup.select("div.place_section._3fSeV > div > ul > li")
    print("찾아진 리뷰 개수 :",len(reviews))
    review_json[gym_name] = []
    if len(reviews)==0 :
      print("리뷰가 없습니다.")
      continue
  except NoSuchElementException:
    print("리뷰가 없습니다")

  print("리뷰 내용 open!")
  try:
    for r in range(len(reviews)):
      review_content = reviews[r].select("div.faZHB > a > span")
      if len(review_content)>0: # 글 리뷰가 없는 경우 방지
        review_json[gym_name].append(review_content[0].getText())
        print("리뷰 내용 :", review_content[0].getText())
        print("-----------------------------------------------------")
  except NoSuchElementException:
    print("리뷰 안찾아짐")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
리뷰 내용 : 24시간 365일 운동할 수 있고, 내부도 유산소, 근력운동 기구 잘 갖춰져있고 시설도 쾌적하게 유지돼서 좋습니다. 추천합니다
-----------------------------------------------------
리뷰 내용 : 여기 완전 운동 맛집입니다 파워랙이 3개라니.... 덤벨 존도 2곳이나 있습니다  오늘도 근성장 조지러 갑니다
-----------------------------------------------------
리뷰 내용 : 지하에 있지만 쾌적합니다. 특히 샤워실 매일 청소해주시는 거 !! 이게 제일 좋아요 :) 
-----------------------------------------------------
리뷰 내용 : 친절한 상담 감사합니다.
학교랑 가까워서 좋네요.
시설도 깔끔합니다. 추천해요!
-----------------------------------------------------
리뷰 내용 : 여기 직원분들도 친절하고 파워렉이 진짜 많아서 좋아요 오리엔테이션도 좋아여
-----------------------------------------------------
리뷰 내용 : 24시간 운영되어 어느때든 운동할 수 있는 점이 큰 장점입니다. 
시설도 괜찮고 트레이너, 데스크 분들도 모두 친절하십니다!
-----------------------------------------------------
리뷰 내용 : 작년 9월부터 꾸준히 다니고 있는데 헬스장이 계속 더 나아지고 있어서 좋습니다!
-----------------------------------------------------
리뷰 내용 : 시설도 깔끔하고 친절하게 상담 해주셔서 고민하다가 등록 했어용~
제발 이번 다이어트는 성공할 수 있길!ㅎㅎ
-----------------------------------------------------
리뷰 내용 : 시설도 좋고 깔끔합니다 그

In [ ]:
import json

file_path = '/content/drive/MyDrive/Colab Notebooks/seongdonggu_review.json'
with open(file_path,'w') as f:
  json.dump(review_json, f, indent="\t", ensure_ascii=False)

In [ ]:
review_df.columns

Index(['company_id', 'review0', 'review1', 'review2', 'review3', 'review4',
       'review5', 'review6', 'review7', 'review8', 'review9'],
      dtype='object')

In [ ]:
review_df.head()

,company_id,review0,review1,review2,review3,review4,review5,review6,review7,review8,review9
0,1150285601,주변에 추천하고 싶은 헬스장입니다. 친절하게 맞아주시고 상담도 자세히 해주셔서 첫인...,재등록합니다. 운동하기 너무 좋아요~ 강력 추천합니다👍👍,디스크로 고생해서 병원 다니다가 여기서 pt 받아봤는데 전보다 훨씬 좋아졌어요. 몸...,PT 선생님들 정말 전문지식 가득하시고 꼼꼼하고 체계적으로 자세 봐주시고 알려주세요...,집 가까운곳에 이렇게 좋은 헬스장이 있어서 좋네요 운동기구도 엄청다양하고 선생님들이...,자세 잘봐주시고 시설도 너무 좋고 운동할 맛이 납니다\n앞으로 뼈를 묻겠습니다!!!,"선생님들도 친절하시고, 기구도 거의 새것이고, 분위기도 너무 좋습니다!! 운동이 하...",오픈한지 얼마 되지 않아 깔끔하고 쾌적하네요^^ 친절하게 상담 해주셔서 고민없이 바...,3개월 열심히 다니고 또 등록했습니다! 선생님들 친절하시며 매우 깨끗합니다!! 좋아...,혼자 운동 하다 좋아서 와이프도 추천하여 PT\n운동 시작 하는대 합리적이고 좋아요
1,272888535,집 근처에 있어서 이용하기 편하네여!!,굳,굿,굿,굿,좋아요,항상 재등록하는 곳이에요 시설도 크고 관리 잘되는 편이구요 요즘 코로나때매 신경 많...,엄청 크고 이근처에선 젤 좋음,친구소개로 운동하게 되었는데 쾌적하고 너무좋네요. 다욧제대로 하겠어요,센터 상태가 청결하고 좋습니다.
2,18788499,좋아요,좋아요,짱짱너무좋아요,,,,,,,
3,32085717,가성비 헬스장! 동네에 있는 곳이라 부담없이 다니기 좋을 것 같아요. 잘 다녀볼게요!,ㅎ,ㅎ,,,,,,,
4,1530150594,처음왔는데 굉장히 친절하시고 등록방법과 시설안내 등등도 자세히 설명해주셨어요. 위치...,기구들도 다양하고 깔끔해서 너무 좋아요! 금액대 너무 좋습니다!!!!,"프리모션, 인바디 맛집 헬스장 분위기도 좋고 기구 대부분은 프리모션 기구로 이루...",사장님이 너무 친절하시구 운동기구도 다양하고 새것 같고 시설도 너무 깔끔합니다ㅎㅎ ...,시설 좋고 위치가 굉장히 마음에 듭니다 👍👍☺️,집 앞 이여서 고민하다가 등록하러 오게됐어용\n친절하고 좋아용~,사장님 친절하시고 기구들도 좋아서 열심히 운동 할수 있을것 같아요. 올 여름에는 당...,너무 친절하시고 기구도 너무 많아요 ㅎㅎ \n열심히 운동 하게 될 것 같은 좋은기분!,우연히 들린 짐아서휘트니스\n봄맞이 할인 이벤트\n열심히 운동해서 몸 만들자,시설이 깨끗하고 운동하기 좋아요


In [ ]:
review_df.to_csv("/content/drive/MyDrive/Colab Notebooks/seongdonggu_review.csv")

## 헬스장 별점 크롤링

In [ ]:
df['grade'] = 100

In [ ]:
import requests

In [ ]:
for i in range(len(df)):
  print(i,"번째 헬스장")
  print(df['loc_name'][i])
  url = 'https://pcmap.place.naver.com/place/'+df['naver_map_url'][i]+'/review/visitor#'
  print(url)
  result = requests.get(url)
  soup = BeautifulSoup(result.text, "html.parser")

  try:
    grade = soup.select("#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div._20Ivz > span._1Y6hi._1A8_M > em")[0]
    df['grade'][i] = float(grade.getText())
    print(grade.getText())
  except IndexError:
    df['grade'][i] = 100
    print("별점이 없습니다.")

0 번째 헬스장
올포유짐
https://pcmap.place.naver.com/place/1150285601/review/visitor#
5
1 번째 헬스장
투엑스휘트니스 옥수점
https://pcmap.place.naver.com/place/272888535/review/visitor#


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


4.92
2 번째 헬스장
올림픽휘트니스
https://pcmap.place.naver.com/place/18788499/review/visitor#


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


별점이 없습니다.
3 번째 헬스장
스타트짐헬스
https://pcmap.place.naver.com/place/32085717/review/visitor#
별점이 없습니다.
4 번째 헬스장
짐아서
https://pcmap.place.naver.com/place/1530150594/review/visitor#
별점이 없습니다.
5 번째 헬스장
라온휘트니스&필라테스 마장역점
https://pcmap.place.naver.com/place/21111585/review/visitor#
별점이 없습니다.
6 번째 헬스장
웰트짐 성수점
https://pcmap.place.naver.com/place/11691712/review/visitor#
4.45
7 번째 헬스장
동우휘트니스
https://pcmap.place.naver.com/place/37309187/review/visitor#
4.05
8 번째 헬스장
시티짐 답십리점
https://pcmap.place.naver.com/place/34346765/review/visitor#
별점이 없습니다.
9 번째 헬스장
포시즌휘트니스 왕십리점
https://pcmap.place.naver.com/place/37638105/review/visitor#
4.5
10 번째 헬스장
팀키스짐
https://pcmap.place.naver.com/place/1061867273/review/visitor#
별점이 없습니다.
11 번째 헬스장
노블휘트니스
https://pcmap.place.naver.com/place/1213139053/review/visitor#
4.67
12 번째 헬스장
고릴라멀티짐 행당점
https://pcmap.place.naver.com/place/1806684520/review/visitor#
4.85
13 번째 헬스장
스포애니 왕십리역점
https://pcmap.place.naver.com/place/1006755563/review/visitor#
3
14 번째 헬스장
센트리얼휘트니스 옥수점
https://

In [ ]:
df.head()

,open,loc_number,loc_address,loc_name,loc_lng,loc_lat,naver_map_url,grade
0,영업중,2298-7071,"서울특별시 성동구 독서당로 272, 상가동 201호 (금호동4가, 금호동 대우아파트)",올포유짐,127.016793,37.545973,1150285601,5.00
1,영업중,02-2299-0044,"서울특별시 성동구 독서당로 166, B201호 (옥수동, 성원상떼뷰)",투엑스휘트니스 옥수점,127.008734,37.540707,272888535,4.92
2,영업중,469-0067,"서울특별시 성동구 뚝섬로 400, 뚝섬쇼핑센타 3층 (성수동2가)",올림픽휘트니스,127.052078,37.538473,18788499,100.00
3,영업중,0507-1349-5118,"서울특별시 성동구 성덕정길 106, 승재빌딩 4층 (성수동2가)",스타트짐헬스,127.052205,37.537100,32085717,100.00
4,영업중,461-9901,"서울특별시 성동구 성수일로 39, 경동빌딩 지층 (성수동1가)",짐아서,127.046208,37.544323,1530150594,100.00


In [ ]:
df.rename(columns={'rating':'grade'},inplace=True)

In [ ]:
df[df['grade']==100].count()

open             45
loc_number       29
loc_address      45
loc_name         45
loc_lng          45
loc_lat          45
naver_map_url    45
grade            45
dtype: int64

## 헬스장 사진 크롤링

In [ ]:
# naver_map_url 바꿔주기
for i in range(len(df)):
  url = df['naver_map_url'][i]
  url = url[36:]
  num = url.index('/')
  url = url[:num]
  df['naver_map_url'][i] = url
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,open,loc_number,loc_address,loc_name,loc_lng,loc_lat,naver_map_url,grade
0,영업중,2298-7071,"서울특별시 성동구 독서당로 272, 상가동 201호 (금호동4가, 금호동 대우아파트)",올포유짐,127.016793,37.545973,1150285601,5.00
1,영업중,02-2299-0044,"서울특별시 성동구 독서당로 166, B201호 (옥수동, 성원상떼뷰)",투엑스휘트니스 옥수점,127.008734,37.540707,272888535,4.92
2,영업중,469-0067,"서울특별시 성동구 뚝섬로 400, 뚝섬쇼핑센타 3층 (성수동2가)",올림픽휘트니스,127.052078,37.538473,18788499,100.00
3,영업중,0507-1349-5118,"서울특별시 성동구 성덕정길 106, 승재빌딩 4층 (성수동2가)",스타트짐헬스,127.052205,37.537100,32085717,100.00
4,영업중,461-9901,"서울특별시 성동구 성수일로 39, 경동빌딩 지층 (성수동1가)",짐아서,127.046208,37.544323,1530150594,100.00


In [ ]:
s = "https://pcmap.place.naver.com/place/"
len(s)

36

In [ ]:
df['img'] = " "

In [ ]:
len("<div class='cb7hz _div' id='ibu_1' style='width:100%;height:100%;background-image:url(")

86

In [ ]:
import requests
for i in range(len(df)):
  print(i,"번째 헬스장")
  print(df['loc_name'][i])
  url = 'https://pcmap.place.naver.com/place/'+str(df['naver_map_url'][i])+'/home'
  print(url)
  result = requests.get(url)
  soup = BeautifulSoup(result.text, "html.parser")

  try:
    img = str(soup.select("#ibu_1")[0])
    num = img.index('>')
    print(img[87:num-3])
    df['img'][i] = img[87:num-3]
  except IndexError:
    df['img'][i] = ' '
    print("사진이 없습니다.")

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


0 번째 헬스장
올포유짐
https://pcmap.place.naver.com/place/1150285601/home
https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220217_91%2F1645085410851HlqS5_JPEG%2F%25BC%25BA%25B1%25D4%25C7%25DC_%25B5%25EE.jpg
1 번째 헬스장
투엑스휘트니스 옥수점
https://pcmap.place.naver.com/place/272888535/home


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210729_15%2F1627553853956IIUr8_JPEG%2Fa4KcwbEbJyCzaJCNpP2x_-ED.jpg
2 번째 헬스장
올림픽휘트니스
https://pcmap.place.naver.com/place/18788499/home
https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20190925_272%2F1569409871094y7Etq_JPEG%2Fcrms7n8jDfTAYU4IEgLqBZ2T.jpeg.jpg
3 번째 헬스장
스타트짐헬스
https://pcmap.place.naver.com/place/32085717/home
https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220221_201%2F16454508635153jB9o_JPEG%2FScreenshot_20220221-223845_Gallery.jpg
4 번째 헬스장
짐아서
https://pcmap.place.naver.com/place/1530150594/home
https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210428_40%2F1619593520488PvHJw_PNG%2FBhapJTgTNDwX9KVs8V5stwZy.png
5 번째 헬스장
라온휘트니스&필라테스 마장역점
https://

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


사진이 없습니다.
8 번째 헬스장
시티짐 답십리점
https://pcmap.place.naver.com/place/34346765/home
사진이 없습니다.
9 번째 헬스장
포시즌휘트니스 왕십리점
https://pcmap.place.naver.com/place/37638105/home
https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220406_196%2F1649244393646JB81U_JPEG%2F%25BD%25C2%25C8%25F1.jpg
10 번째 헬스장
팀키스짐
https://pcmap.place.naver.com/place/1061867273/home
https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220215_256%2F1644898852420gYa6F_JPEG%2FKakaoTalk_20220215_131359018.jpg
11 번째 헬스장
노블휘트니스
https://pcmap.place.naver.com/place/1213139053/home
https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210206_63%2F1612596631235qsgt8_PNG%2F-KUQJkknmvMUz9YpbWNEGSh8.png
12 번째 헬스장
고릴라멀티짐 행당점
https://pcmap.place.naver.com/place/1806684520/home
https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_s

In [ ]:
df.head()

,open,loc_number,loc_address,loc_name,loc_lng,loc_lat,naver_map_url,grade,photo
0,영업중,2298-7071,"서울특별시 성동구 독서당로 272, 상가동 201호 (금호동4가, 금호동 대우아파트)",올포유짐,127.016793,37.545973,1150285601,5.00,https://search.pstatic.net/common/?autoRotate=...
1,영업중,02-2299-0044,"서울특별시 성동구 독서당로 166, B201호 (옥수동, 성원상떼뷰)",투엑스휘트니스 옥수점,127.008734,37.540707,272888535,4.92,https://search.pstatic.net/common/?autoRotate=...
2,영업중,469-0067,"서울특별시 성동구 뚝섬로 400, 뚝섬쇼핑센타 3층 (성수동2가)",올림픽휘트니스,127.052078,37.538473,18788499,100.00,https://search.pstatic.net/common/?autoRotate=...
3,영업중,0507-1349-5118,"서울특별시 성동구 성덕정길 106, 승재빌딩 4층 (성수동2가)",스타트짐헬스,127.052205,37.537100,32085717,100.00,https://search.pstatic.net/common/?autoRotate=...
4,영업중,461-9901,"서울특별시 성동구 성수일로 39, 경동빌딩 지층 (성수동1가)",짐아서,127.046208,37.544323,1530150594,100.00,https://search.pstatic.net/common/?autoRotate=...


## 헬스장 주소 크롤링

In [ ]:
df['address2'] = ""

In [ ]:
import requests
for i in range(len(df)):
  print(i,"번째 헬스장")
  print(df['name'][i])
  url = 'https://pcmap.place.naver.com/place/'+str(df['id'][i])+'/home'
  print(url)
  result = requests.get(url)
  soup = BeautifulSoup(result.content.decode('utf-8','replace'), "html.parser")

  try:
    address = soup.select("._2yqUQ")[0].getText()
    print(address)
    df['address2'][i] = address
  except IndexError:
    continue

0 번째 헬스장
올포유짐
https://pcmap.place.naver.com/place/1150285601/home
서울 성동구 독서당로 270 대우아파트상가 201호
1 번째 헬스장
투엑스휘트니스 옥수점
https://pcmap.place.naver.com/place/272888535/home


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


서울 성동구 독서당로 166 성원상떼뷰 1F/B1/B2
2 번째 헬스장
올림픽휘트니스
https://pcmap.place.naver.com/place/18788499/home
서울 성동구 뚝섬로 400 뚝섬쇼핑센타
3 번째 헬스장
스타트짐헬스
https://pcmap.place.naver.com/place/32085717/home
서울 성동구 성덕정길 106
4 번째 헬스장
짐아서
https://pcmap.place.naver.com/place/1530150594/home
서울 성동구 성수일로 39 경동빌딩 B1
5 번째 헬스장
라온휘트니스&필라테스 마장역점
https://pcmap.place.naver.com/place/21111585/home
서울 성동구 마장로 302 지하 1, 2층
6 번째 헬스장
웰트짐 성수점
https://pcmap.place.naver.com/place/11691712/home
서울 성동구 연무장5가길 21 홍일빌딩 지하2층
7 번째 헬스장
동우휘트니스
https://pcmap.place.naver.com/place/37309187/home
서울 성동구 천호대로 296
8 번째 헬스장
시티짐 답십리점
https://pcmap.place.naver.com/place/34346765/home
서울 성동구 천호대로 308 인암빌딩 지하2층
9 번째 헬스장
포시즌휘트니스 왕십리점
https://pcmap.place.naver.com/place/37638105/home
서울 성동구 왕십리광장로 17 왕십리역사 6층 포시즌휘트니스
10 번째 헬스장
팀키스짐
https://pcmap.place.naver.com/place/1061867273/home
서울 성동구 동호로 100 두산종합상가 4층
11 번째 헬스장
노블휘트니스
https://pcmap.place.naver.com/place/1213139053/home
서울 성동구 상원길 63 쌍용아파트 쌍용상가 A동 B01호
12 번째 헬스장
고릴라멀티짐 행당점
https://pcmap.place

## 헬스장 시간 크롤링
https://pcmap.place.naver.com/place/38241100/home <br>
https://pcmap.place.naver.com/place/1463857548/home

In [ ]:
time_df = pd.DataFrame()
time_df['company_id'] = df['id']

In [ ]:
time_df['mon_open'] = " "
time_df['mon_close'] = " "
time_df['tue_open'] = " "
time_df['tue_close'] = " "
time_df['wed_open'] = " "
time_df['wed_close'] = " "
time_df['thu_open'] = " "
time_df['thu_close'] = " "
time_df['fri_open'] = " "
time_df['fri_close'] = " "
time_df['sat_open'] = " "
time_df['sat_close'] = " "
time_df['sun_open'] = " "
time_df['sun_close'] = " "

In [ ]:
time_df.head()

,company_id,mon_open,mon_close,tue_open,tue_close,wed_open,wed_close,thu_open,thu_close,fri_open,fri_close,sat_open,sat_close,sun_open,sun_close
0,1150285601,06:00,23:00,06:00,23:00,06:00,23:00,06:00,23:00,06:00,23:00,08:00,20:00,10:00,18:00
1,272888535,06:00,23:30,06:00,23:30,06:00,23:30,06:00,23:30,06:00,23:30,09:00,21:00,09:00,18:00
2,18788499,06:00,23:30,06:00,23:30,06:00,23:30,06:00,23:30,06:00,23:30,09:00,20:00,10:00,17:00
3,32085717,05:00,23:50,05:00,23:50,05:00,23:50,05:00,23:50,05:00,23:50,05:00,21:00,10:00,18:00
4,1530150594,06:00,23:00,06:00,23:00,06:00,23:00,06:00,23:00,06:00,23:00,08:00,17:00,09:00,


In [ ]:
time_df.to_csv("/content/drive/MyDrive/Colab Notebooks/seongdonggu_time.csv")

In [ ]:
time_df.to_csv("/content/drive/MyDrive/Colab Notebooks/seongdonggu_time2.csv")

In [ ]:
time_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/seongdonggu_time.csv")
time_df = time_df[['company_id','mon_open','mon_close','tue_open','tue_close','wed_open','wed_close','thu_open','thu_close','fri_open','fri_close','sat_open','sat_close','sun_open','sun_close']]

In [ ]:
time_idx = {"월":['mon_open','mon_close'],'화':['tue_open','tue_close'],'수':['wed_open','wed_close'],'목':['thu_open','thu_close'],'금':['fri_open','fri_close'],'토':['sat_open','sat_close'],'일':['sun_open','sun_close']}

In [ ]:
time_idx.keys()

dict_keys(['월', '화', '수', '목', '금', '토', '일'])

In [ ]:
time_df.iloc[7,:]

company_id    37309187
mon_open         06:00
mon_close        01:00
tue_open         06:00
tue_close        01:00
wed_open         06:00
wed_close        01:00
thu_open         06:00
thu_close        01:00
fri_open         06:00
fri_close        01:00
sat_open         06:00
sat_close        21:00
sun_open         06:00
sun_close        21:00
Name: 7, dtype: object

In [ ]:
for i in range(len(df)):
  print(i,"번째 헬스장")
  print(df['name'][i])
  url = 'https://pcmap.place.naver.com/place/'+str(df['id'][i])+'/home'
  print(url)
  driver.get(url)
  time.sleep(5)

  try: # 더보기 클릭
    time.sleep(1)
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
    time.sleep(3)
    driver.find_element(By.CSS_SELECTOR, ".place_section_content > ul ._2KHqk ._1h3B_ > a").send_keys(Keys.ENTER)
    time.sleep(3)
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
    time.sleep(1)
  except NoSuchElementException:
    print('-더보기 버튼 모두 클릭 완료-')

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
  time.sleep(1)
  # result = requests.get(url)
  # soup = BeautifulSoup(result.content.decode('utf-8','replace'), "html.parser")

  try:
    time_section = soup.select(".place_section_content > ul ._2KHqk ._1h3B_ > a")[0]
    time_days = time_section.select("div > div > span > span") [1:]
    time_days.pop()
    time_content = time_section.select("div > div > span > div")

    if time_days[0].getText() =='매일': # 예외처리
        l = time_content[0].getText().split(' - ')
        for k in time_idx.keys():
          time_df[time_idx[k][0]][i], time_df[time_idx[k][1]][i] = l
        print("매일", time_content[0].getText())
        continue

    for j in range(len(time_days)): # 월화수목금토일 나눠져 있는 경우 (대부분)
      d = time_days[j].getText()
      t = time_content[j].getText()
      print(d, t)

      if '0' in t:
        # l = t.split(' - ')
        # time_df[time_idx[d][0]][i], time_df[time_idx[d][1]][i] = l
        open_idx = t.index(":")
        close_idx = len(t)- 1-t[::-1].index(":")
        open_time = t[open_idx-2:open_idx+3]
        close_time = t[close_idx-2:close_idx+3]
        time_df[time_idx[d][0]][i], time_df[time_idx[d][1]][i] = open_time, close_time

    # for t in time_days:
    #   print(t.getText())
    # for t in time_content:
    #   print(t.getText())
    # # span = time_section.select("div > a > div:nth-of-type(2) > div > span > span")
    # #print(time_section)
  except IndexError:
    time_string = time_section.select("div > div > span")
    time_string.pop()
    for t in time_string:
      t = t.getText()
      print(t)
      open_idx = t.find(":")
      close_idx = len(t)- 1-t[::-1].find(":")
      open_time = t[open_idx-2:open_idx+3] if open_idx!=-1 else " "
      close_time = t[close_idx-2:close_idx+3] if close_idx!=-1 else  " "
      if "평일" in t:
        weekday = ['월','화','수','목','금']
        for w in weekday:
          time_df[time_idx[w][0]][i],time_df[time_idx[w][1]][i] = open_time, close_time
      if "주말" in t:
        weekend = ['토','일']
        for w in weekend:
          time_df[time_idx[w][0]][i],time_df[time_idx[w][1]][i] = open_time, close_time
      day_idx = t.find("요일")
      if day_idx!=-1:
        yoil = t[day_idx-1]
        time_df[time_idx[yoil][0]][i], time_df[time_idx[yoil][1]][i] = open_time, close_time
      # if "월요일" in t:
      #   time_df[time_idx["월"][0]][i], time_df[time_idx["월"][1]][i] = open_time, close_time
      # if '화요일' in t:
      #   time_df[time_idx["화"][0]][i], time_df[time_idx["화"][1]][i] = open_time, close_time
      # if "수요일" in t:
      #   time_df[time_idx["수"][0]][i], time_df[time_idx["수"][1]][i] = open_time, close_time
      # if '목요일' in t:
      #   time_df[time_idx["목"][0]][i], time_df[time_idx["목"][1]][i] = open_time, close_time
      # if "토요일" in t:
      #   time_df[time_idx["토"][0]][i], time_df[time_idx["토"][1]][i] = open_time, close_time
      # if '월요일' in t or '화요일' in t or "수요일" in t or "목요일" in t or "금요일" in t or "토요일" in t or'일요일' in t:
      #   time_df[time_idx["일"][0]][i], time_df[time_idx["일"][1]][i] = open_time, close_time

0 번째 헬스장
올포유짐
https://pcmap.place.naver.com/place/1150285601/home


/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


목 06:00 - 23:00
금 06:00 - 23:00
토 08:00 - 20:00
일 10:00 - 18:00
월 06:00 - 23:00
화 06:00 - 23:00
수 06:00 - 23:00
1 번째 헬스장
투엑스휘트니스 옥수점
https://pcmap.place.naver.com/place/272888535/home
평일 06:00 - 23:30 
토요일 09:00 - 21:00 
일요일 09:00 - 18:00 매월 첫째, 셋째 휴관
2 번째 헬스장
올림픽휘트니스
https://pcmap.place.naver.com/place/18788499/home


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


평일 06:00 - 23:30 
토요일 09:00 - 20:00 
일요일 10:00 - 17:00 1.3 주 일요일만!
3 번째 헬스장
스타트짐헬스
https://pcmap.place.naver.com/place/32085717/home
목 05:00 - 23:50
금 05:00 - 23:50
토 05:00 - 21:00
일 10:00 - 18:00
월 05:00 - 23:50
화 05:00 - 23:50
수 05:00 - 23:50
4 번째 헬스장
짐아서
https://pcmap.place.naver.com/place/1530150594/home


...

목 06:00 - 23:00
금 06:00 - 23:00
토 08:00 - 17:00
일 정기휴무 (매주 일요일)
월 06:00 - 23:00
화 06:00 - 23:00
수 06:00 - 23:00
5 번째 헬스장
라온휘트니스&필라테스 마장역점
https://pcmap.place.naver.com/place/21111585/home
목 06:00 - 24:00
금 06:00 - 24:00
토 10:00 - 20:00
일(7/24) 정기휴무 (매달 2, 4, 5번째 일요일)
월 06:00 - 24:00
화 06:00 - 24:00
수 06:00 - 24:00
6 번째 헬스장
웰트짐 성수점
https://pcmap.place.naver.com/place/11691712/home
목 06:30 - 23:30
금 06:30 - 23:30
토 09:00 - 18:00
일 정기휴무 (매주 일요일)
월 06:30 - 23:30
화 06:30 - 23:30
수 06:30 - 23:30
7 번째 헬스장
동우휘트니스
https://pcmap.place.naver.com/place/37309187/home
평일 06:00 - 01:00 (새벽1시)
토요일 06:00 - 21:00 일요일휴관
8 번째 헬스장
시티짐 답십리점
https://pcmap.place.naver.com/place/34346765/home
목 06:00 - 24:00
금 06:00 - 24:00
토 10:00 - 20:00
일 10:00 - 20:00
월 06:00 - 24:00
화 06:00 - 24:00
수 06:00 - 24:00
9 번째 헬스장
포시즌휘트니스 왕십리점
https://pcmap.place.naver.com/place/37638105/home
평일 06:00 - 24:00 
토요일 06:00 - 21:00 
공휴일 09:00 - 21:00 매월 셋째 주 일요일 정기 휴무
10 번째 헬스장
팀키스짐
https://pcmap.place.naver.com/place/1061867273/home

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


매일 00:00 - 24:00
16 번째 헬스장
스타트짐
https://pcmap.place.naver.com/place/12032777/home
평일 06:00 - 24:00 
토요일 10:00 - 18:00 
일요일 12:00 - 17:00 둘째,넷째 일요일 정기휴관
공휴일 12:00 - 17:00 
17 번째 헬스장
파크짐 행당점
https://pcmap.place.naver.com/place/34287354/home
매일 00:00 - 24:00
18 번째 헬스장
파크짐 행당점
https://pcmap.place.naver.com/place/34287354/home
매일 00:00 - 24:00
19 번째 헬스장
MSC 운동센터
https://pcmap.place.naver.com/place/36105399/home
목 10:00 - 23:00
금 10:00 - 23:00
토 09:00 - 18:00
일 정기휴무 (매주 일요일)
월 10:00 - 23:00
화 10:00 - 23:00
수 10:00 - 23:00
20 번째 헬스장
에코휘트니스
https://pcmap.place.naver.com/place/36418609/home
목 06:00 - 23:00
금 06:00 - 23:00
토 09:30 - 19:00
일 09:30 - 18:00
월 06:00 - 23:00
화 06:00 - 23:00
수 06:00 - 23:00
21 번째 헬스장
건강과땀 한양대점
https://pcmap.place.naver.com/place/405152522/home
목 06:30 - 22:00
금 06:30 - 22:00
토 09:00 - 17:00
일 정기휴무 (매주 일요일)
월 06:30 - 22:00
화 06:30 - 22:00
수 06:30 - 22:00
22 번째 헬스장
인피니티퍼스널트레이닝
https://pcmap.place.naver.com/place/37233073/home
목 05:30 - 23:00
금 05:30 - 23:00
토 08:00 - 16

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


목 06:00 - 24:00
금 06:00 - 24:00
토 10:00 - 18:00
일 정보없음
월 06:00 - 24:00
화 06:00 - 24:00
수 06:00 - 24:00
27 번째 헬스장
바른몸 PT STUDIO
https://pcmap.place.naver.com/place/823249514/home
목 08:00 - 23:00
금 08:00 - 23:00
토 정보없음
일 정기휴무 (매주 일요일)
월 08:00 - 23:00
화 08:00 - 23:00
수 08:00 - 23:00
28 번째 헬스장
피트니스에이치
https://pcmap.place.naver.com/place/1278400640/home
목 06:00 - 23:00
금 06:00 - 23:00
토 09:00 - 19:00
일(7/24) 정기휴무 (매달 2, 4번째 일요일)
월 06:00 - 23:00
화 06:00 - 23:00
수 06:00 - 23:00
29 번째 헬스장
투엑스휘트니스 센트라스점
https://pcmap.place.naver.com/place/73553317/home
목 06:00 - 23:30
금 06:00 - 23:30
토 09:00 - 21:00
일 09:00 - 18:00
월 06:00 - 23:30
화 06:00 - 23:30
수 06:00 - 23:30
30 번째 헬스장
커브스 금호클럽
https://pcmap.place.naver.com/place/32489668/home
목 10:00 - 20:00
금 10:00 - 20:00
토 10:00 - 13:00
일 정보없음
월 10:00 - 20:00
화 10:00 - 20:00
수 10:00 - 20:00
31 번째 헬스장
테라피티 뚝섬역점
https://pcmap.place.naver.com/place/1578353949/home
목 07:00 - 23:00
금 07:00 - 23:00
토 08:00 - 14:00
일 정기휴무 (매주 일요일)
월 07:00 - 23:00
화 07:00 - 23:0

In [ ]:
print(time_idx[time_days[0].getText()][0])

wed_open


In [ ]:
time_content[i].getText().split(' - ')

['06:00', '23:00']

## 헬스장 키워드 리뷰 크롤링
https://pcmap.place.naver.com/place/36418609/review/visitor# <br>
이런점이 좋았어요 칸이 없는 경우도 있음

In [ ]:
keyword_review_df = pd.DataFrame()

In [ ]:
keyword_review_df['company_id'] = df['id']

In [ ]:
keyword_review_df['kind'] = 0
keyword_review_df['tidy'] = 0
keyword_review_df['various'] = 0
keyword_review_df['wide'] = 0
keyword_review_df['reasonable'] = 0
keyword_review_df['pose'] = 0
keyword_review_df['advice'] = 0
keyword_review_df['parking'] = 0
keyword_review_df['class'] = 0
keyword_review_df['GX'] =0
keyword_review_df['care'] =0
keyword_review_df['teacher'] =0

In [ ]:
keyword_review_df['view'] =0
keyword_review_df['shower'] =0
keyword_review_df['excessive'] =0

In [ ]:
keyword_review_df.head()

,company_id,kind,tidy,various,wide,reasonable,pose,advice,parking,class,GX
0,1150285601,0,0,0,0,0,0,0,0,0,0
1,272888535,0,0,0,0,0,0,0,0,0,0
2,18788499,0,0,0,0,0,0,0,0,0,0
3,32085717,0,0,0,0,0,0,0,0,0,0
4,1530150594,0,0,0,0,0,0,0,0,0,0


In [ ]:
keyword_review_idx = {'"시설이 깔끔해요"':"tidy",'"친절해요"':"kind",'"가격이 합리적이에요"':'reasonable',
                      '"기구가 다양해요"':'various','"상담이 자세해요"':'advice','"공간이 넓어요"':'wide',
                      '"자세를 꼼꼼히 봐주세요"':'pose','"수업이 체계적이에요"':'class','"주차하기 편해요"':'parking','"GX프로그램이 다양해요"':'GX',
                      '"선생님 실력이 좋아요"':'teacher','"맞춤 케어를 잘해줘요"':'care','"뷰가 좋아요"':'view','"샤워실이 잘 되어있어요"':'shower',
                      '"과도한 권유가 없어요"':'excessive'}

In [ ]:
keyword_review_df.iloc[75,:]

company_id    38696246
kind                 0
tidy                 0
various              0
wide                 0
reasonable           0
pose                 0
advice               0
parking              0
class                0
GX                   0
care                 0
teacher              0
view                 0
shower               0
excessive            0
Name: 75, dtype: int64

In [ ]:
keyword_review_df.to_csv("/content/drive/MyDrive/Colab Notebooks/seongdonggu_comment.csv")

In [ ]:
for i in range(len(df)):
  print(i,"번째 헬스장")
  gym_name = df['name'][i]
  print("헬스장 이름 :",gym_name)
  thisurl = url = 'https://pcmap.place.naver.com/place/'+str(df['id'][i])+'/review/visitor#'
  print(thisurl)
  driver.get(thisurl)
  time.sleep(5)

  try: # 더보기 클릭
    time.sleep(1)
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
    time.sleep(3)
    #driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._2m9np > div > div > div._3ljhX > a').click() # 더보기
    driver.find_element(By.CLASS_NAME,"_12z-G").send_keys(Keys.ENTER)
    time.sleep(3)
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
    time.sleep(1)
  except:
    print('-더보기 버튼 모두 클릭 완료-')

  html = driver.page_source
  soup = BeautifulSoup(html, 'lxml')
  time.sleep(1)

  print("li 찾기 open!")
  try:
    #li = soup.select("#app-root > div > div > div > div:nth-of-type(7) > div:nth-of-type(2) > div.place_section._2m9np > div > div > div._3ljhX > ul > li")
    li = soup.select("div.place_section._2m9np > div > div > div._3ljhX > ul > li")
    # if len(li)==0:
    #   print("im here")
    #   li = soup.select("#app-root > div > div > div > div:nth-of-type(6) > div > div > div > div > div._3ljhX > ul > li")
    for l in li:
      keyword = l.select("div > span")
      for idx in range(0,len(keyword),2):
        review = keyword[idx].getText()
        num = int(keyword[idx+1].getText()[13:])
        print(review, num)
        keyword_review_df[keyword_review_idx[review]][i] = num
    # review_json[gym_name] = []
    # if len(reviews)==0:
    #   reviews = soup.select("#app-root > div > div > div > div:nth-of-type(7) > div > div.place_section._3fSeV > div > ul > li") # 이런 점이 좋아요 없을 경우
    #   if len(reviews)==0 :
    #     print("리뷰가 없습니다.")
    #     reviews = []
    #     continue
  except NoSuchElementException:
    print("리뷰가 없습니다")

  # print("리뷰 내용 open!")
  # try:
  #   for r in range(len(reviews)):
  #     review_content = reviews[r].select("div.faZHB > a > span")
  #     if len(review_content)>0: # 글 리뷰가 없는 경우 방지
  #       review_json[gym_name].append(review_content[0].getText())
  #       print("리뷰 내용 :", review_content[0].getText())
  #       print("-----------------------------------------------------")
  #   for r in range(10-len(reviews)):
  #     review_json[gym_name].append(" ")
  # except NoSuchElementException:
  #   print("리뷰 안찾아짐")

0 번째 헬스장
헬스장 이름 : 올포유짐
https://pcmap.place.naver.com/place/1150285601/review/visitor#
li 찾기 open!
"시설이 깔끔해요" 335
"친절해요" 335
"가격이 합리적이에요" 235
"기구가 다양해요" 226
"상담이 자세해요" 210
"공간이 넓어요" 79
"자세를 꼼꼼히 봐주세요" 79
"수업이 체계적이에요" 61
"주차하기 편해요" 29
"GX프로그램이 다양해요" 20
1 번째 헬스장
헬스장 이름 : 투엑스휘트니스 옥수점
https://pcmap.place.naver.com/place/272888535/review/visitor#
-더보기 버튼 모두 클릭 완료-
li 찾기 open!
2 번째 헬스장
헬스장 이름 : 올림픽휘트니스
https://pcmap.place.naver.com/place/18788499/review/visitor#
-더보기 버튼 모두 클릭 완료-
li 찾기 open!
3 번째 헬스장
헬스장 이름 : 스타트짐헬스
https://pcmap.place.naver.com/place/32085717/review/visitor#
-더보기 버튼 모두 클릭 완료-
li 찾기 open!
4 번째 헬스장
헬스장 이름 : 짐아서
https://pcmap.place.naver.com/place/1530150594/review/visitor#
li 찾기 open!
"친절해요" 17
"시설이 깔끔해요" 16
"기구가 다양해요" 15
"상담이 자세해요" 10
"가격이 합리적이에요" 10
"공간이 넓어요" 8
"자세를 꼼꼼히 봐주세요" 7
"수업이 체계적이에요" 4
"GX프로그램이 다양해요" 2
"주차하기 편해요" 1
5 번째 헬스장
헬스장 이름 : 라온휘트니스&필라테스 마장역점
https://pcmap.place.naver.com/place/21111585/review/visitor#
-더보기 버튼 모두 클릭 완료-
li 찾기 open!
6 번째 헬스장
헬스장 이름 : 웰트짐 성수점
http

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/datasets/서초구_company_plus2.csv")
df.head()

,Column1,id,name,img,grade,lat,lng,phoneNumber,address,naver_map_url,address2
0,0,0,서초헬스크럽,NaN,4.57,37.503811,127.021293,542-5457,"서울특별시 서초구 사평대로 349 (반포동,2층)",1744103267,서울 서초구 서초중앙로 15 상가 B1 슈퍼빌스포츠클럽
1,1,1,한전아트센터스포츠클럽,https://search.pstatic.net/common/?autoRotate=...,4.40,37.485605,127.028079,2055-1331,서울특별시 서초구 효령로72길 60 (서초동),1223657721,서울 서초구 효령로72길 60
2,2,2,우성헬스 2,https://search.pstatic.net/common/?autoRotate=...,NaN,37.488122,127.014931,582-1817,"서울특별시 서초구 서초중앙로 72 (서초동,지하1층)",1226621811,서울 서초구 효령로77길 34 3층 310호
3,3,3,코아텔휘트니스클럽,https://search.pstatic.net/common/?autoRotate=...,4.50,37.493962,127.027916,6288-3301,"서울특별시 서초구 강남대로53길 11 (서초동,지하2층)",12922948,서울 서초구 강남대로53길 11 삼성쉐르빌2 지하2층 전층
4,4,4,대한퍼스널트레이닝,https://search.pstatic.net/common/?autoRotate=...,NaN,37.510009,127.019980,515-2108,"서울특별시 서초구 사평대로55길 139 (반포동,미성빌딩4층)",37264113,서울 강남구 테헤란로5길 36 1층


13

In [ ]:
for i in range(2):
  print(i,"번째 헬스장")
  print(df['loc_name'][i])
  url = 'https://pcmap.place.naver.com/place/'+str(df['naver_map_url'][i])+'/review/visitor#'
  print(url)
  result = requests.get(url)
  soup = BeautifulSoup(result.text, "html.parser")

  try:
    # keyword = soup.select("#app-root > div > div > div > div:nth-of-type(7) > div:nth-of-type(2) > div.place_section._2m9np > div > div > div._3ljhX > ul > li")
    keyword = soup.findAll("li", attrs={'class':'X4uex'})
    print(keyword)
    print(len(keyword))
  except IndexError:
    print("없습니다")

0 번째 헬스장
올포유짐
https://pcmap.place.naver.com/place/1150285601/review/visitor#
[<li class="X4uex"><div class="_1cQoC" style="width:78.11764705882352%"></div><div class="_2B33i"><img alt="" class="_2iZo0" height="22" src="https://ssl.pstatic.net/static/pup/emoji/sparkles20220119222028.png" width="22"/><span class="_10uv0">"<!-- -->ìì¤ì´ ê¹ëí´ì<!-- -->"</span><span class="_3IFxe"><span class="place_blind">ì´ í¤ìëë¥¼ ì íí ì¸ì</span>332</span></div></li>, <li class="X4uex"><div class="_1cQoC" style="width:77.88235294117646%"></div><div class="_2B33i"><img alt="" class="_2iZo0" height="22" src="https://ssl.pstatic.net/static/pup/emoji/beating_heart20220119222223.png" width="22"/><span class="_10uv0">"<!-- -->ì¹ì í´ì<!-- -->"</span><span class="_3IFxe"><span class="place_blind">ì´ í¤ìëë¥¼ ì íí ì¸ì</span>331</span></div></li>, <li class="X4uex"><div class="_1cQoC" style="width:54.8235294117647%"></div><div class="_2B33i"><img alt="" class="_2iZo0" heig

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/seongdonggu_comment.csv")
df = df.drop(df.columns[[0]], axis=1)
df.head()

,company_id,kind,tidy,various,wide,reasonable,pose,advice,parking,class,GX,care,teacher,view,shower,excessive
0,1150285601,335,335,226,79,235,79,210,29,61,20,0,0,0,0,0
1,272888535,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,18788499,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,32085717,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1530150594,17,16,15,8,10,7,10,1,4,2,0,0,0,0,0


# 휴지통

In [ ]:
for i in range(len(df)):
  print(i,"번째 헬스장")
  # gym_name = df['loc_name'][i]
  # print(gym_name)
  # print(len(review_json[gym_name]))
  # print(review_num_json[gym_name])
  # if len(review_json[gym_name]) == review_num_json[gym_name]:
  #   continue
  thisurl = df['naver_map_url'][i]
  print(thisurl)
  driver.get(thisurl)
  time.sleep(5)
  cnt = 0

  # while True:
  #   try: # 더보기 클릭
  #     cnt+=1
  #     print(cnt, end = ' ')
  #     time.sleep(1)
  #     driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
  #     time.sleep(3)
  #     #driver.find_element(By.XPATH,"//*[@id='app-root']/div/div/div/div[7]/div[2]/div[3]/div[2]/a").click()
  #     driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._3fSeV > div._2kAri > a').click() # 더보기
  #     time.sleep(3)
  #     driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
  #     time.sleep(1)
  #   except:
  #     print('-더보기 버튼 모두 클릭 완료-')
  #     break



  html = driver.page_source
  soup = BeautifulSoup(html, 'lxml')
  time.sleep(1)

  try: # 헬스장 이름 수정
    gym_name_naver = soup.find_all('span',attrs = {'class':'_3XamX'})[0]
    print(gym_name_naver.getText())
    df['loc_name'][i] = gym_name_naver.getText()
  except NoSuchElementException:
    print("헬스장 이름 찾을 수 없습니다.")

  gym_name = df['loc_name'][i]
  print("헬스장 이름 :",gym_name)
  review_num_json[gym_name] = 0

  print("리뷰 개수 open!")
  try: # 리뷰 개수
    review_num = soup.find_all('span', attrs = {'class':'place_section_count'}) # 리뷰 개수
    print(review_num.parent.getText())
    review_num_json[gym_name] = int(review_num[0].getText())
    print("리뷰 개수 :", review_num)
  except IndexError:
    print("리뷰 개수 찾을 수 없습니다.")
    review_num_json[gym_name] = 0
    review_json[gym_name] = []
    continue

  print("리뷰 확인 open!")
  try:
    reviews = soup.select("#app-root > div > div > div > div:nth-of-type(7) > div:nth-of-type(2) > div.place_section._3fSeV > div > ul > li")
    print("찾아진 리뷰 개수 :",len(reviews))
    review_json[gym_name] = []
    if len(reviews)==0:
      reviews = soup.select("#app-root > div > div > div > div:nth-of-type(7) > div > div.place_section._3fSeV > div > ul > li") # 이런 점이 좋아요 없을 경우
      if len(reviews)==0 :
        print("리뷰가 없습니다.")
        reviews = []
        continue
  except NoSuchElementException:
    print("리뷰가 없습니다")

  print("리뷰 내용 open!")
  try:
    for r in range(len(reviews)):
      review_content = reviews[r].select("div.faZHB > a > span")
      if len(review_content)>0: # 글 리뷰가 없는 경우 방지
        review_json[gym_name].append(review_content[0].getText())
        print("리뷰 내용 :", review_content[0].getText())
        print("-----------------------------------------------------")
    for r in range(10-len(reviews)):
      review_json[gym_name].append(" ")
  except NoSuchElementException:
    print("리뷰 안찾아짐")

  # print("리뷰 내용 open!")
  # try:
  #   for r in range(len(reviews)):
  #     review_id = reviews[r].select("div._1orw_ > a._3VOZq > div._1vou-")
  #     review_content = reviews[r].select("div.faZHB > a > span")
  #     if len(review_id)>0: # 아이디가 없는 경우 방지
  #       if len(review_content)>0: # 글 리뷰가 없는 경우 방지
  #         review_json[gym_name].append({'review_id':review_id[0].getText(),'review_content':review_content[0].getText()})
  #         print("리뷰 아이디 :", review_id[0].getText())
  #         print("리뷰 내용 :", review_content[0].getText())
  #         print("-----------------------------------------------------")
  # except NoSuchElementException:
  #   print("리뷰 안찾아짐")

In [ ]:
for i in range(len(df)):
  print(i,"번째 헬스장")
  print(df['loc_name'][i])
  url = 'https://pcmap.place.naver.com/place/'+df['naver_map_url'][i]+'/photo'
  print(url)
  result = requests.get(url)
  soup = BeautifulSoup(result.text, "html.parser")

  try:
    img = soup.findAll('img')[0]
    print(img)
    # df['photo'][i] = img
    # print(img.src)
  except IndexError:
    df['photo'][i] = ' '
    print("사진이 없습니다.")

0 번째 헬스장
올포유짐
https://pcmap.place.naver.com/place/1150285601/photo
<img alt="" class="xx5Gm" height="28" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f48_48&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220217_91%2F1645085410851HlqS5_JPEG%2F%25BC%25BA%25B1%25D4%25C7%25DC_%25B5%25EE.jpg" width="28"/>
1 번째 헬스장
투엑스휘트니스 옥수점
https://pcmap.place.naver.com/place/272888535/photo
<img alt="" class="xx5Gm" height="28" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f48_48&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210729_15%2F1627553853956IIUr8_JPEG%2Fa4KcwbEbJyCzaJCNpP2x_-ED.jpg" width="28"/>
2 번째 헬스장
올림픽휘트니스
https://pcmap.place.naver.com/place/18788499/photo
<img alt="" class="xx5Gm" height="28" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f48_48&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20190925_272%2F1569409871094y7Etq_JPEG%2Fcrms7n8jDfTAYU4IEgLqBZ2T.jpeg.jpg" width="28"/>
3 번째 헬스

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


사진이 없습니다.
8 번째 헬스장
시티짐 답십리점
https://pcmap.place.naver.com/place/34346765/photo
<img alt="" class="xx5Gm" height="28" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f48_48&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200505_285%2F1588660831598Sdwsx_JPEG%2FE47k5oCOgFI4KEMykE6eAmlr.jpg" width="28"/>
9 번째 헬스장
포시즌휘트니스 왕십리점
https://pcmap.place.naver.com/place/37638105/photo
<img alt="" class="xx5Gm" height="28" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f48_48&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220406_196%2F1649244393646JB81U_JPEG%2F%25BD%25C2%25C8%25F1.jpg" width="28"/>
10 번째 헬스장
팀키스짐
https://pcmap.place.naver.com/place/1061867273/photo
<img alt="" class="xx5Gm" height="28" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f48_48&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220215_256%2F1644898852420gYa6F_JPEG%2FKakaoTalk_20220215_131359018.jpg" width="28"/>
11 번째 헬스장
노블휘트니

KeyboardInterrupt: ignored